### UI 代码
* button：点击后打开选择目录
* text：展示选中的目录路径

In [40]:
import tkinter as tk
from tkinter import filedialog
import ipywidgets as widgets
from IPython.display import HTML


def select_directory():
    # 初始化 Tkinter 主窗口
    root = tk.Tk()
    root.withdraw()  # 隐藏主窗口

    # 打开文件选择对话框
    folder_selected = filedialog.askdirectory()

    # 关闭 Tkinter 主窗口
    root.destroy()

    return folder_selected
def select_file():
    # 初始化 Tkinter 主窗口
    root = tk.Tk()
    root.withdraw()  # 隐藏主窗口

    # 打开文件选择对话框
    folder_selected = filedialog.askopenfilename()

    # 关闭 Tkinter 主窗口
    root.destroy()

    return folder_selected


button_dir = widgets.Button(
    description='选择解析目录',
    # disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='选择目录，目录下所有文件将解析',
)

button_file = widgets.Button(
    description='选择解析文件',
    # disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='选择的文件将被解析',
)

text = widgets.Text(
    value='',
    placeholder='解析路径',
    # description='选中的目录地址:',
    disabled=False
)
def btn_disable():
    # button_file.disabled = not button_file.disabled
    # button_dir.disabled = not button_dir.disabled
    return

def dir_btn_click(b):
    btn_disable()
    text.value = select_directory()
    btn_disable()

def file_btn_click(b):
    btn_disable()
    text.value = select_file()
    btn_disable()

button_dir.on_click(dir_btn_click)
button_file.on_click(file_btn_click)

# widgets.GridBox([button_dir,button_file,text],layout=widgets.Layout(grid_template_columns="repeat(3, 160px)"))


### 解析逻辑
根据文件类型，使用特定的解析方式，按正则表达式规则做替换。

In [29]:
import re

# js 替换
# 将类名转换为小驼峰形式
def to_upperCase(match):
    return match.group(1).capitalize()


def camelCaseClassName(str):
    return re.sub(r"[-_]([a-z1-9]*)", to_upperCase, str)


def replace_class_name(match):
    class_name = match.group(1)
    return f"className={{styles.{camelCaseClassName(class_name)}}}"


def replace_import(match):
    return 'import styles from "./index.module.less"'


# 转换 less 样式文件
def replace_less(match):
    groups = match.groups()

    return groups[0] + camelCaseClassName(groups[1])


js_patterns_and_replacements = {
    r'className=[\"\']([^"\']*)[\"\']': replace_class_name,
    r"import [\"\']\./index\.less[\"\']": replace_import,
}

less_patterns_and_replacements = {r"([.#])([\w-]+)": replace_less}


"""
定义输出方式:
n: 创建新文件
w: 覆盖

n 模型下，文件默认命名，例：index.tsx -> index.new.tsx

"""

transform_config = [
    {
        "suffix":['.jsx','.tsx','.js'],
        "patterns_and_replacements": js_patterns_and_replacements,
        "output":{
            "mode":'w',
            "name":''
        }
    },
    {
        "suffix":['.less'],
        "patterns_and_replacements": less_patterns_and_replacements,
        "output":{
            "mode":'n',
            "name":'[name].module[suffix]' # 默认[name][suffix]
        }
        
    }
]

In [36]:
from pathlib import Path
import re


# 读取文件
def readFileSync(file_path: Path):
    return file_path.read_text()


def writeFileSync(file_path: Path, content: str):
    return file_path.write_text(content)


# 生成
def generate_file(file_path: Path, config, content):
    output = config["output"]
    mode = output["mode"]
    if mode == "n":
        file_name = output.get("name", "new")

        placeholders = {
            "[name]": file_path.stem, 
            "[suffix]": file_path.suffix
        }

        for placeholder, value in placeholders.items():
            file_name = file_name.replace(placeholder, value)

        file_path = file_path.parent / file_name
        writeFileSync(file_path, content)
    elif mode == "w":
        writeFileSync(file_path, content)
    return


# 转换react文件
def transform_file(file_path: Path, config):
    file_content = readFileSync(file_path)
    if not file_content:
        return

    transformed_content = file_content
    for pattern, replacement in config["patterns_and_replacements"].items():
        transformed_content = re.sub(pattern, replacement, transformed_content)
    generate_file(file_path, config, transformed_content)
    # print(transformed_content)


# 处理文件
def processFile(file_path: Path):
    for item in transform_config:
        if file_path.suffix in item["suffix"]:
            transform_file(file_path.resolve(), item)
    return


def start(path):
    dir_path = Path(path)

    if not path:
        return
    
    if dir_path.is_file():
        processFile(Path(dir_path))
        return
    
    if dir_path.is_dir():
        for file_path in dir_path.rglob('*'):
            if file_path.is_file() and 'node_modules' not in file_path.parts:
                processFile(Path(file_path))
    return

In [41]:
widgets.GridBox([button_dir,button_file,text],layout=widgets.Layout(grid_template_columns="repeat(3, 160px)"))

GridBox(children=(Button(button_style='success', description='选择解析目录', style=ButtonStyle(), tooltip='选择目录，目录下所…

In [42]:

path = text.value
print(path)
start(path)

/Users/liepin/liepin-project/cnpm-nest-react-core-fe/packages/nest-react-resume-pc/src/components
